# RecDP LLM - rough score deduplication

Remove similar data by calculating the rough score. Using python library [rouge-score](https://pypi.org/project/rouge-score/)


### use case:
* Expect Input format: a folder of *.jsonl.
* Expect Output format: a folder of *.jsonl after reduction.


# Get started

## Install pyrecdp and dependencies

In [4]:
! DEBIAN_FRONTEND=noninteractive apt-get install -y openjdk-8-jre
! pip install pyrecdp --pre
# ! pip install 'git+https://github.com/intel/e2eAIOK.git#egg=pyrecdp&subdirectory=RecDP'

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-8-jre is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


## 2. prepare your own data

In [2]:
%mkdir -p /content/test_data
%cd /content/test_data
file_list = ['tiny_c4_sample.jsonl']
file_list += [f"https://raw.githubusercontent.com/intel/e2eAIOK/main/RecDP/tests/data/llm_data/{i}" for i in file_names]
!wget -P /content/test_data {" ".join(file_list)}

/content/test_data
--2023-11-10 07:49:21--  http://tiny_c4_sample.jsonl/
Resolving proxy-prc.intel.com (proxy-prc.intel.com)... 10.240.252.16
Connecting to proxy-prc.intel.com (proxy-prc.intel.com)|10.240.252.16|:912... connected.
Proxy request sent, awaiting response... 500 Internal Server Error
2023-11-10 07:49:21 ERROR 500: Internal Server Error.

--2023-11-10 07:49:21--  https://raw.githubusercontent.com/intel/e2eAIOK/main/RecDP/tests/data/llm_data/tiny_c4_sample.jsonl
Connecting to proxy-prc.intel.com (proxy-prc.intel.com)|10.240.252.16|:912... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1062126 (1.0M) [text/plain]
Saving to: ‘/content/test_data/tiny_c4_sample.jsonl’

tiny_c4_sample.json 100%[===================>]   1.01M   922KB/s    in 1.1s    

2023-11-10 07:49:23 (922 KB/s) - ‘/content/test_data/tiny_c4_sample.jsonl’ saved [1062126/1062126]

FINISHED --2023-11-10 07:49:23--
Total wall clock time: 2.2s
Downloaded: 1 files, 1.0M in 1.1s (922 KB/s)


## 3. fuzzy deduplicate (seperate detection and reduction)

In [3]:
! ls /content/test_data

tiny_c4_sample.jsonl


### 3.1 PIPELINE based API

In [2]:
from pyrecdp.LLM import TextPipeline, ResumableTextPipeline
from pyrecdp.primitives.operations import *

pipeline = ResumableTextPipeline()
# optional to enable or not enable statistics.
# enable_statistics helps to show summary info
# not enable_statistics provides better performance
pipeline.enable_statistics()

# start to add ops to pipeline
ops = [
    JsonlReader("/content/test_data/"),
    RougeScoreDedup(text_key='text', max_ratio=0.7, batch_size=20),
]
pipeline.add_operations(ops)
# pipeline.plot()

2023-11-10 07:51:50.416 | WARNING  | pyrecdp.LLM.TextPipeline:enable_statistics:214 - Enabling this option will result in a decrease in execution speed


{0: {'children': None, 'op': 'DatasetReader', 'config': {}},
 1: {'children': [0], 'op': 'JsonlReader', 'config': {'input_dir': '/content/test_data/'}},
 2: {'children': [1], 'op': 'RougeScoreDedup', 'config': {'text_key': 'text', 'max_ratio': 0.7, 'batch_size': 20, 'score_store_path': 'RougeScorefiltered.parquet'}}}

In [3]:
ret = pipeline.execute()

[DatasetReader, PerfileSourcedJsonlReader, RougeScoreDedup, PerfileParquetWriter]
Will assign 48 cores and 412162 M memory for spark


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/10 07:51:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


per core memory size is 8.385 GB and shuffle_disk maximum capacity is 8589934592.000 GB
execute with spark for global tasks started ...
DatasetReader
2023-11-10 07:51:58.406 | INFO     | pyrecdp.LLM.TextPipeline:execute:365 - DatasetReader: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
execute with spark for global tasks took 0.003309955000077025 sec
PerfileSourcedJsonlReader


ResumableTextPipeline, current on tiny_c4_sample.jsonl:   0%|          | 0/1 [00:00<?, ?it/s]

tiny_c4_sample.jsonl
RougeScoreDedup
statistics_decorator spark


...

Round 22 took 2.671257287999879 sec
generate_connected_components => duplicates started ...



100%|██████████| 27/27 [00:00<00:00, 16104.41it/s]


2023-11-10 07:58:32.416 | INFO     | pyrecdp.primitives.operations.text_compare_dedup:process_spark:130 - Finally detected duplicated num_samples is 12
generate_connected_components => duplicates took 0.2964779440001166 sec
2023-11-10 07:58:32.930 | INFO     | pyrecdp.LLM.TextPipeline:execute:404 - RougeScoreDedup: A total of 449 rows of data were processed, using 388.623343706131 seconds, A duplication list containing 437 found, around 97.32739420935413% of total data, Sampled, duplication preview:     id_1  id_2     id_pair                                    similarity_left  \
0    351    19   19 :: 351  After tipping 25 tokens in a day, you'll be ab...   
1    201     4    4 :: 201  After tipping 25 tokens in a day, you'll be ab...   
2    351     4    4 :: 351  After tipping 25 tokens in a day, you'll be ab...   
3     19     4     4 :: 19  After tipping 25 tokens in a day, you'll be ab...   
4    320     4    4 :: 320  After tipping 25 tokens in a day, you'll be ab...   
5    320 

ResumableTextPipeline, current on tiny_c4_sample.jsonl: 100%|██████████| 1/1 [06:28<00:00, 388.99s/it]

2023-11-10 07:58:32.939 | INFO     | pyrecdp.LLM.TextPipeline:execute:409 - Completed! ResumableTextPipeline will not return dataset, please check ResumableTextPipeline_output_20231110075154 for verification.


In [ ]:
pipeline.plot()
del pipeline